# As a statistical interaction network model

Fixed parameters of the model are:
 - an _internal state spce / response space / local microstates_, $\sigma_{\mathit int} = \{ 0, 1\}$
 - an _external state space_, $\sigma_{\mathit ext} = [ 0, 2\pi]$
 - position distribution of elements / components / statistical units, $X_i \sim WExp(\lambda)$
 - a connection function, 
$p:\sigma_{\mathit ext} \times \sigma_{\mathit ext} \rightarrow [0,1]: (\theta_i, \theta_j) \mapsto \min \Big\{1, k - \frac{k}{2\pi a}d(\theta_i, \theta_j) \Big\}$,

where $d(.,.)$ is the "_periodic distance_", i.e. $d(\theta_i, \theta_j) = \min {\Big\{ \big|\theta_i-\theta_j\big|, 2\pi-\big|\theta_i-\theta_j\big|} \Big\}$

# As an Ising model with a "_random lattice_"

---
**NEW**

I think the Ising model makes more sense for the "line graph", i.e. where the edges represent components and nodes define interactions.

---

Given the three parameters $N$, $a$ and $\beta$ we can construct a "_random lattice_" $\Lambda$ via:
$$\langle ij \rangle \Leftrightarrow d(\Theta_i, \Theta_j) < 2\pi a \, , \text{where } \Theta_k \sim \mathit WExp (\lambda)$$

The two parameters $\lambda$ and $\beta$ can be transformed into each other via
 - $\lambda = 1 / \tan\Big(\frac{\pi}{2}\beta\Big)$
 - $\beta = \frac{2}{\pi} \cdot \arctan(1/\lambda)$

# As a Generalized random geometric graph (RGG)

The connection fucntion is taken from the "_Quasi unit disk model_" via
$$H(d) =
    \begin{cases}
      1, & \text{if}\ d \leq d_- \\
      \frac{d_+ - d}{d_+ - d_-}, & \text{if}\ d_- \leq d \leq d_+ \\
      0, & \text{if}\ d \geq d_+
    \end{cases} \, ,$$ 
    
where $d_- = 2\pi a (1-1/k)$ and $d_+ = 2$. 

The small $d$ expansion is the constant $1$ (similar to a random geometric graph model with a "hard" cutoff).

### References
- Dettmann, Carl P., and Orestis Georgiou. "Random geometric graphs with general connection functions." Physical Review E 93.3 (2016): 032313.
- Kuhn, F., R. Wattenhofer, and A. Zollinger. "Proceedings of the 2003 Joint Workshop on Foundations of Mobile Computing." (2003): 69-78.
- Gao, Deyun, et al. "Hop-distance relationship analysis with quasi-UDG model for node localization in wireless sensor networks." EURASIP Journal on Wireless Communications and Networking 2011.1 (2011): 1-11

# Parameter identifications

$\lambda = \dfrac{1}{\tan\Big(\frac{\pi}{2}\beta\Big)}$

# Walking guide

First we initialize a random drawing of the positions on the circle.

In [69]:
import numpy as np
import pandas as pd
import ringity as rng
import scipy.stats as ss
import plotly.express as px
import plotly.graph_objects as go

In [75]:
N = 2**10
delay = 0.

frozen_distn1 = rng.classes._distns.get_frozen_wrappedexpon(arg = 0.001, argtype = 'delay')
frozen_distn2 = rng.classes._distns.get_frozen_wrappedexpon(arg = 0.5, argtype = 'delay')
frozen_distn3 = rng.classes._distns.get_frozen_wrappedexpon(arg = 0.999, argtype = 'delay')

periodic_positions1 = rng.classes.network_model.PeriodicPositionGenerator(N, distn_arg = frozen_distn1)
periodic_positions2 = rng.classes.network_model.PeriodicPositionGenerator(N, distn_arg = frozen_distn2)
periodic_positions3 = rng.classes.network_model.PeriodicPositionGenerator(N, distn_arg = frozen_distn3)

In [76]:
plot_data_short = pd.DataFrame({'pos1': periodic_positions1.positions, 
                                'pos2': periodic_positions2.positions, 
                                'pos3': periodic_positions3.positions})

plot_data_long = plot_data_short.melt()

In [77]:
px.histogram(plot_data_long, x = 'value', color='variable', marginal="rug")

In [87]:
fig = go.Figure()
fig.update_xaxes(range=[-0.5, 2*np.pi])
fig.add_trace(go.Histogram(x=plot_data_short.pos2))
fig.add_trace(go.Histogram(x=plot_data_short.pos3))
fig.add_trace(go.Histogram(x=plot_data_short.pos1))

# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)
fig.show()

In [112]:
df

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
0,Afghanistan,Asia,1952,28.801,8425333,779.445314,AFG,4
1,Afghanistan,Asia,1957,30.332,9240934,820.853030,AFG,4
2,Afghanistan,Asia,1962,31.997,10267083,853.100710,AFG,4
3,Afghanistan,Asia,1967,34.020,11537966,836.197138,AFG,4
4,Afghanistan,Asia,1972,36.088,13079460,739.981106,AFG,4
...,...,...,...,...,...,...,...,...
1699,Zimbabwe,Africa,1987,62.351,9216418,706.157306,ZWE,716
1700,Zimbabwe,Africa,1992,60.377,10704340,693.420786,ZWE,716
1701,Zimbabwe,Africa,1997,46.809,11404948,792.449960,ZWE,716
1702,Zimbabwe,Africa,2002,39.989,11926563,672.038623,ZWE,716


In [111]:
df = px.data.gapminder()
px.scatter(df, x="gdpPercap", y="lifeExp", animation_frame="year", animation_group="country",
           size="pop", color="continent", hover_name="country",
           log_x=True, size_max=55, range_x=[100,100000], range_y=[25,90])

In [89]:
print(fig)

Figure({
    'data': [{'opacity': 0.75,
              'type': 'histogram',
              'x': array([0.40605832, 1.84265342, 0.67782646, ..., 3.8309856 , 1.31216848,
                          0.08117447])},
             {'opacity': 0.75,
              'type': 'histogram',
              'x': array([3.66138402, 1.98211995, 5.65162165, ..., 1.75533865, 4.44640694,
                          1.56232434])},
             {'opacity': 0.75,
              'type': 'histogram',
              'x': array([2.75256008e-03, 6.64095384e-04, 1.19345361e-04, ..., 1.61587126e-03,
                          1.47487886e-05, 2.51410843e-03])}],
    'layout': {'barmode': 'overlay', 'template': '...', 'xaxis': {'range': [-0.5, 6.283185307179586]}}
})


# Spielwiese